In [ ]:
import fatbot as fb
import torch as tt
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def plotresX(path, color):
    font = {
        #'family': 'serif',
            'weight': 'normal',
            'size': 16,
            }
    eval_d = np.load( path[0] )
    eval_dx = np.load( path[1] )

    eval_zip = { k:eval_d[k] for k in eval_d.keys() }
    print(eval_zip.keys())
    eval_d.close()
    xr = eval_zip['timesteps']
    res = eval_zip['results'].mean(axis=-1)
    eps = eval_zip['ep_lengths'].mean(axis=-1)
    print(xr.shape, res.shape, eps.shape)

    eval_zipx = { k:eval_dx[k] for k in eval_dx.keys() }
    print(eval_zipx.keys())
    eval_dx.close()
    xrx = eval_zipx['timesteps']
    resx = eval_zipx['results'].mean(axis=-1) 
    epsx = eval_zipx['ep_lengths'].mean(axis=-1)
    print(xrx.shape, resx.shape, epsx.shape)
    
    
    fig,axs = plt.subplots(2, 1, figsize=(12,12), sharex=True)

    titles=['(a) base-policy', '(b) auxiliary-policy']
    labels=['base-policy', 'auxiliary-policy']
    r= [res, resx]
    x= [xr, xrx]
    for i in range(2):
        ax=axs[i]
        ax.grid(axis='both')
        ax.set_title(titles[i], fontdict=font)

        ax.set_ylabel('Episodic Return', fontdict=font)
        
        ax.scatter(x[i], r[i], color=color[i])
        ax.plot(x[i], r[i], color=color[i], label=labels[i])
        #ax.legend()
    axs[1].set_xlabel('Training steps')#, fontdict=font)
    plt.tight_layout()
    plt.show()

    return fig

In [ ]:
def plotrestx(path):
    font = {
        #'family': 'serif',
            'weight': 'normal',
            'size': 16,
            }
    linewidth=1.5
    eval_d = np.load( path )

    eval_zip = { k:eval_d[k] for k in eval_d.keys() }
    print(eval_zip.keys())
    eval_d.close()
    tehist = eval_zip['th']
    sehist = eval_zip['sh']
    switch_history = eval_zip['wh']
    print(tehist.shape, sehist.shape, switch_history.shape)

        
    fig, ax = plt.subplots(2, 1, figsize=(12,12), sharex=True)
    #fig.suptitle(f'Rewards @ Episodes {len(tehist)}')
    ax[0].set_ylabel('Reward',fontdict=font)
    ax[1].set_ylabel('Return',fontdict=font)
    ax[1].set_xlabel('Simulation time-step',fontdict=font)
    for episode,episode_reward_history in enumerate(tehist):
        max_reward_at = sehist[episode]
        switch_history_at = switch_history[episode]
        episode_reward_history=np.array(episode_reward_history)
        #fig.suptitle(f'Episode: {episode+1}')
        rewh, reth = episode_reward_history[:,0], episode_reward_history[:,1]
        #ax[0].plot(rewh, color='tab:blue', linewidth=0.8) # label='Reward',
        #ax[0].scatter([max_reward_at],rewh[max_reward_at], color='tab:blue')
        #ax[0].scatter([switch_history_at],rewh[switch_history_at], color='tab:red')
        
        # ax[0].annotate('Policy-Switch',xy=(switch_history_at, rewh[switch_history_at]),
        #                 xytext= (switch_history_at+0.05, rewh[switch_history_at]+0.05),
        #                 arrowprops=dict(arrowstyle="->", linewidth=0.7, linestyle='solid' , color='black')) # distance vector)


        xr = np.arange(len(rewh))
        ax[0].plot(xr[0:switch_history_at], rewh[0:switch_history_at],  color='tab:green', linewidth=linewidth) # label='Return',
        ax[0].plot(xr[switch_history_at:], rewh[switch_history_at:],  color='tab:blue', linewidth=linewidth) # label='Return',
        ax[0].plot([xr[switch_history_at-1], xr[switch_history_at]], 
                   [rewh[switch_history_at-1], rewh[switch_history_at]], color='tab:blue', linewidth=linewidth)
        ax[0].scatter([switch_history_at-1],rewh[switch_history_at-1], color='tab:red', marker='o')

        xr = np.arange(len(reth))
        ax[1].plot(xr[0:switch_history_at], reth[0:switch_history_at],  color='tab:green', 
                    linewidth=linewidth, label='base-policy') # label='Return',
        ax[1].plot(xr[switch_history_at:], reth[switch_history_at:],  color='tab:blue', 
                    linewidth=linewidth, label='auxi-policy') # label='Return',
        ax[1].plot([xr[switch_history_at-1], xr[switch_history_at]], 
                   [reth[switch_history_at-1], reth[switch_history_at]], color='tab:blue', linewidth=linewidth)
        ax[1].scatter([switch_history_at-1],reth[switch_history_at-1], color='tab:red', marker='o', 
                      label='policy-switch step')

        #ax[1].scatter()
    ax[0].grid(axis='both')
    ax[1].grid(axis='both')
    ax[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25),
            fancybox=True, shadow=False, ncol=3, fontsize=17)
        

    #fig.savefig(pjs(save_states, f'{model_type}_test_per_episode.png'))
    plt.show()

    # fig, ax = plt.subplots(2, 1, figsize=(12,6))
    # fig.suptitle(f'Test Results')
    # ax[0].plot(test_history[:,0], label='Steps', color='tab:purple')
    # ax[1].plot(test_history[:,1], label='Return', color='tab:green')
    # ax[0].legend()
    # ax[1].legend()
    # #fig.savefig(pjs(save_states, f'{model_type}_test_all_episodes.png'))
    # plt.show()
    return fig

# plot raining and testing 

In [ ]:
res_path = '__results__/'
outpath = os.path.join(res_path, 'resplot')

dbs = ['db6', 'db8', 'db10',]# 'db12']
exper = [
    ('D', 'CD'),
    ('P', 'CP'),
    ('S', 'CS'),
    ('D', 'CC'),
    ('P', 'CC'),
    ('S', 'CC'),
]

plot_testing=True

os.makedirs(outpath, exist_ok=True)

for db in dbs:
    
    #db='db6'

    for modelx,config in exper:
        #modelx = 'D'
        #config=f'C{modelx}'

        pbase = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/base/evaluations.npz')
        paux = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/auxi/evaluations.npz')
        pfinal = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/results/final_results.npz')



        if plot_testing:
            fig=plotrestx(pfinal)
            fig.savefig(os.path.join(outpath, f'test_{db}_config_{config}_{modelx}'), dpi=300)
        else:
                
            fig = plotresX((pbase, paux), ('tab:green', 'tab:blue'))
            fig.savefig(os.path.join(outpath, f'train_{db}_config_{config}_{modelx}'),  dpi=300)

# How many steps

In [ ]:
res_paths = ['__results__',]
episodes=20
dbs = ['db6', 'db8','db10',]# 'db12']
exper = [
    ('D', 'CD'),
    ('P', 'CP'),
    ('S', 'CS'),
    ('D', 'CC'),
    ('P', 'CC'),
    ('S', 'CC'),
]
font = {
    #'family': 'serif',
        'weight': 'normal',
        'size': 16,
        }
lables=dict(
    DCD = 'Distributed - predefined target',
    PCP = 'Packed - predefined target',
    SCS = 'Scattered - predefined target',
    DCC = 'Distributed - undefined target',
    PCC = 'Packed - undefined target',
    SCC = 'Scattered - undefined target',
)
divs = 1/len(exper)
fig=plt.figure(figsize=(12,6), dpi=300)
plt.grid(axis='y')
plt.xticks(np.arange(4, 12, 2))
plt.xlim(4, 12)
plt.xlabel('Swarm size', fontdict=font)
plt.ylabel('Average steps for gathering', fontdict=font)


tx = np.array([6,8,10])
for res_path in res_paths:
   # outpath = os.path.join(res_path, 'resplot')
    print(f'\n{res_path=}')

    conk = [[] for _ in exper]
    for db in dbs:
        #db='db6'
        npz=np.load(fb.pj(f'{res_path}/{db}_{episodes}.npz'))
        for k in npz.keys(): print(k)
        for i,(d,c) in enumerate(exper):
            conk[i].append(np.mean(npz[f'{db}_config_{c}_{d}'],axis=0)[-1])

        npz.close()
    for i,(d,c) in enumerate(exper):
        plt.bar(tx-divs*i+((len(exper)-1)*0.5*divs), conk[i], divs-0.03, label = lables[f'{d}{c}'] )#f'{d}_{c}')


plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
        fancybox=True, shadow=False, ncol=len(exper)/3, fontsize=17)
plt.show()
fig.savefig('steps.png', dpi=300)

# ARCHIVE

In [ ]:
raise StopIteration

# states view

In [ ]:
kwxy = dict(x=50, y=50, figsize=(4,4))
for pb in [pbase_state_fi, pbase_state_ff, pbase_state_bi, pbase_state_bf]:
    s = tt.load(pb)
    _=s.render(**kwxy)

In [ ]:
kwxy = dict(x=50, y=50, figsize=(4,4))
for pb in [paux_state_fi, paux_state_ff, paux_state_bi, paux_state_bf]:
    s = tt.load(pb)
    _=s.render(**kwxy)

In [ ]:
        pbase_state_fi = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/base/results/0_final_model_initial')
        pbase_state_bi = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/base/results/0_best_model_initial')
        pbase_state_ff = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/base/results/0_final_model_final')
        pbase_state_bf = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/base/results/0_best_model_final')

        paux_state_fi = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/auxi/results/0_final_model_initial')
        paux_state_bi = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/auxi/results/0_best_model_initial')
        paux_state_ff = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/auxi/results/0_final_model_final')
        paux_state_bf = fb.pj(f'{res_path}/model_{db}_config_{config}_{modelx}/auxi/results/0_best_model_final')





# results plot - training

In [ ]:
def plotres(path, alias='base', color='tab:green'):
    eval_d = np.load( path )

    eval_zip = { k:eval_d[k] for k in eval_d.keys() }
    print(eval_zip.keys())
    eval_d.close()
    xr = eval_zip['timesteps']
    res = eval_zip['results'].mean(axis=-1)
    eps = eval_zip['ep_lengths'].mean(axis=-1)
    print(xr.shape, res.shape, eps.shape)


    fig = plt.figure(figsize=(12,6))

    plt.grid(axis='both')
    plt.title(f'Training result for {alias} policy')

    plt.ylabel('Episodic Return')
    plt.xlabel('Training steps')
    plt.scatter(xr, res, color=color)
    plt.plot(xr, res, color=color)

    plt.show()
    #fig.savefig('eval.png')



    # fig = plt.figure(figsize=(12,6))

    # plt.grid(axis='both')

    # plt.ylabel('Episode Length')
    # plt.xlabel('Training steps')
    # plt.title(f'Training result for {alias} policy')
    # plt.scatter(xr, eps, color='tab:purple')
    # plt.plot(xr, eps, color='tab:purple')

    # plt.show()
def plotresx(path, color):
    eval_d = np.load( path[0] )
    eval_dx = np.load( path[1] )

    eval_zip = { k:eval_d[k] for k in eval_d.keys() }
    print(eval_zip.keys())
    eval_d.close()
    xr = eval_zip['timesteps']
    res = eval_zip['results'].mean(axis=-1)
    eps = eval_zip['ep_lengths'].mean(axis=-1)
    print(xr.shape, res.shape, eps.shape)

    eval_zipx = { k:eval_dx[k] for k in eval_dx.keys() }
    print(eval_zipx.keys())
    eval_dx.close()
    xrx = eval_zipx['timesteps']+xr[-1]
    resx = eval_zipx['results'].mean(axis=-1) 
    resx+=( res[-1]-resx[0])
    
    epsx = eval_zipx['ep_lengths'].mean(axis=-1)
    print(xrx.shape, resx.shape, epsx.shape)
    
    
    fig = plt.figure(figsize=(12,12))

    plt.grid(axis='both')
    plt.title(f'Training result')

    plt.ylabel('Episodic Return')
    plt.xlabel('Training steps')
    plt.scatter(xr, res, color=color[0])
    plt.plot(xr, res, color=color[0])
    plt.scatter(xrx, resx, color=color[1])
    plt.plot(xrx, resx, color=color[1])
    plt.show()
    #fig.savefig('eval.png')



    # fig = plt.figure(figsize=(12,6))

    # plt.grid(axis='both')

    # plt.ylabel('Episode Length')
    # plt.xlabel('Training steps')
    # plt.title(f'Training result for {alias} policy')
    # plt.scatter(xr, eps, color='tab:purple')
    # plt.plot(xr, eps, color='tab:purple')

    # plt.show()


In [ ]:
plotres(pbase, alias='base', color='tab:green')

In [ ]:
plotres(paux, alias='auxi', color='tab:blue')

In [ ]:
plotresx((pbase, paux), ('tab:green', 'tab:blue'))

In [ ]:
fig = plotresX((pbase, paux), ('tab:green', 'tab:blue'))
fig.savefig(f'fig_{db}_config_{config}_{modelx}',  dpi=300)

# results plot - testing

In [ ]:
def plotrest(path):
    eval_d = np.load( path )

    eval_zip = { k:eval_d[k] for k in eval_d.keys() }
    print(eval_zip.keys())
    eval_d.close()
    tehist = eval_zip['th']
    sehist = eval_zip['sh']
    switch_history = eval_zip['wh']
    print(tehist.shape, sehist.shape, switch_history.shape)

        
    fig, ax = plt.subplots(2, 1, figsize=(12,6))
    #fig.suptitle(f'Rewards @ Episodes {len(tehist)}')
    ax[0].set_ylabel('Reward')
    ax[1].set_ylabel('Return')
    ax[1].set_xlabel('Simulation time-step')
    for episode,episode_reward_history in enumerate(tehist):
        max_reward_at = sehist[episode]
        switch_history_at = switch_history[episode]
        episode_reward_history=np.array(episode_reward_history)
        #fig.suptitle(f'Episode: {episode+1}')
        rewh, reth = episode_reward_history[:,0], episode_reward_history[:,1]
        ax[0].plot(rewh, color='tab:blue', linewidth=0.8) # label='Reward',
        ax[0].scatter([max_reward_at],rewh[max_reward_at], color='tab:blue')
        ax[0].scatter([switch_history_at],rewh[switch_history_at], color='tab:red')
        ax[0].grid(axis='both')
        # ax[0].annotate('Policy-Switch',xy=(switch_history_at, rewh[switch_history_at]),
        #                 xytext= (switch_history_at+0.05, rewh[switch_history_at]+0.05),
        #                 arrowprops=dict(arrowstyle="->", linewidth=0.7, linestyle='solid' , color='black')) # distance vector)

        ax[1].plot(reth,  color='tab:green', linewidth=0.8) # label='Return',
        ax[1].scatter([max_reward_at],reth[max_reward_at], color='tab:green')
        ax[1].scatter([switch_history_at],reth[switch_history_at], color='tab:red', label='Policy-Switch')
        ax[1].grid(axis='both')
        
    ax[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25),
          fancybox=True, shadow=False, ncol=2)
    #fig.savefig(pjs(save_states, f'{model_type}_test_per_episode.png'))
    plt.show()

    # fig, ax = plt.subplots(2, 1, figsize=(12,6))
    # fig.suptitle(f'Test Results')
    # ax[0].plot(test_history[:,0], label='Steps', color='tab:purple')
    # ax[1].plot(test_history[:,1], label='Return', color='tab:green')
    # ax[0].legend()
    # ax[1].legend()
    # #fig.savefig(pjs(save_states, f'{model_type}_test_all_episodes.png'))
    # plt.show()

In [ ]:
plotrest(pfinal)